<a href="https://colab.research.google.com/github/roychowdary1597/Airline-Passenger-Satisfaction/blob/main/Model_Training_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.impute import SimpleImputer

from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,InputLayer
from keras import regularizers

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import keras_tuner as kt

import joblib
import os

In [5]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Deep Learning ANN/satisfaction.xlsx')

In [6]:
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/Deep Learning ANN"
MODEL_DIR = os.path.join(BASE_DIR, "models")
TUNER_DIR = os.path.join(BASE_DIR, "tuner_logs")

In [7]:
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TUNER_DIR, exist_ok=True)

In [8]:
df.head()

,id,satisfaction_v2,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,11112,satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,...,2,3,3,0,3,5,3,2,0,0.0
1,110278,satisfied,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,...,2,3,4,4,4,2,3,2,310,305.0
2,103199,satisfied,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,...,2,2,3,3,4,4,4,2,0,0.0
3,47462,satisfied,Female,Loyal Customer,60,Personal Travel,Eco,623,0,0,...,3,1,1,0,1,4,1,3,0,0.0
4,120011,satisfied,Female,Loyal Customer,70,Personal Travel,Eco,354,0,0,...,4,2,2,0,2,4,2,5,0,0.0


In [9]:
df.drop(columns='id',axis=1,inplace=True)

In [10]:
# Renaming Columns for our fesability
df = df.rename(columns={'satisfaction_v2':'satisfaction', 'Gender':'gender', 'Customer Type':'customer_type',
                        'Age':'age', 'Type of Travel':'travel_type', 'Class':'class', 'Flight Distance':'distance',
                        'Seat comfort':'seat_comfort', 'Departure/Arrival time convenient':'dep_val_time_convenient',
                        'Food and drink':'food_drink', 'Gate location':'gate','Inflight wifi service':'wifi_service',
                        'Inflight entertainment':'entertainment','Online support':'online_support',
                        'Ease of Online booking':'online_booking_service','On-board service':'onboard_service',
                        'Leg room service':'leg_room_service','Baggage handling':'baggage_handling',
                        'Checkin service':'checkin_service','Cleanliness':'cleanliness','Online boarding':'online_boarding',
                        'Departure Delay in Minutes':'departure_delay_minutes','Arrival Delay in Minutes':'arrival_delay_minutes'})

In [11]:
df.shape

(129880, 23)

In [12]:
# Dropping nulls
df = df.dropna(axis=0)

In [13]:
df.shape #shape after dropping the nulls

(129487, 23)

In [14]:
df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})

In [15]:
df.head()

,satisfaction,gender,customer_type,age,travel_type,class,distance,seat_comfort,dep_val_time_convenient,food_drink,...,online_support,online_booking_service,onboard_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay_minutes,arrival_delay_minutes
0,1,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,...,2,3,3,0,3,5,3,2,0,0.0
1,1,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,...,2,3,4,4,4,2,3,2,310,305.0
2,1,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,...,2,2,3,3,4,4,4,2,0,0.0
3,1,Female,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,...,3,1,1,0,1,4,1,3,0,0.0
4,1,Female,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,...,4,2,2,0,2,4,2,5,0,0.0


In [16]:
#Splitting X and y
X = df.drop(columns=["satisfaction"])
y = df["satisfaction"]

In [17]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

In [18]:
categorical_features

['gender', 'customer_type', 'travel_type', 'class']

In [19]:
numerical_features

['age',
 'distance',
 'seat_comfort',
 'dep_val_time_convenient',
 'food_drink',
 'gate',
 'wifi_service',
 'entertainment',
 'online_support',
 'online_booking_service',
 'onboard_service',
 'leg_room_service',
 'baggage_handling',
 'checkin_service',
 'cleanliness',
 'online_boarding',
 'departure_delay_minutes',
 'arrival_delay_minutes']

In [20]:
print("Data shape:", df.shape)
print("Feature shape:", X.shape)
print("Target shape:", y.shape)

Data shape: (129487, 23)
Feature shape: (129487, 22)
Target shape: (129487,)


In [21]:
# Trainvalset and test set

X_trainval,X_test,y_trainval,y_test = train_test_split(X,y,test_size = 0.2,
                                                       stratify=y,
                                                       random_state=42)

In [22]:
print("shape of data",X_trainval.shape,X_test.shape,y_trainval.shape,y_test.shape)

shape of data (103589, 22) (25898, 22) (103589,) (25898,)


In [23]:
# train and validation

X_train,X_val,y_train,y_val = train_test_split(X_trainval,y_trainval,test_size = 0.2,
                                                       stratify=y_trainval,
                                                       random_state=13)

In [24]:
print("shape of data",X_train.shape,X_val.shape,y_train.shape,y_val.shape)

shape of data (82871, 22) (20718, 22) (82871,) (20718,)


In [25]:
numeric_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

In [26]:
categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

In [27]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [28]:
# Fit on TRAIN only (no leakage)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'distance', 'seat_comfort',
                                  'dep_val_time_convenient', 'food_drink',
                                  'gate', 'wifi_service', 'entertainment',
                                  'online_support', 'online_booking_service',
                                  'onboard_service', 'leg_room_service',
                                  'baggage_handling', 'checkin_service',
                                  'cleanliness', 'online_boarding',
                                  'departure_delay_minutes',
                                  'arrival_delay_minutes']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['gender', 'customer_type', 'travel_type',
                                  'class'])])

In [29]:
# Transform splits
X_train_transformed = preprocessor.transform(X_train)
X_val_transformed   = preprocessor.transform(X_val)
X_test_transformed  = preprocessor.transform(X_test)

In [30]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras import Sequential, layers, regularizers, initializers

In [31]:
def build_model(hp):
    model = Sequential()
    model.add(layers.InputLayer(shape=(X_train_transformed.shape[1],)))

    units = hp.Int("units", 32, 256, step=32, default=128)
    dropout = hp.Float("dropout", 0.0, 0.5, step=0.05, default=0.25)
    l1_val = hp.Float("l1", 1e-5, 1e-3, sampling="log", default=1e-3)
    init = hp.Choice("init", ["he_normal", "glorot_uniform"], default="he_normal")

    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                units,
                activation="relu",
                kernel_initializer=initializers.get(init),
                kernel_regularizer=regularizers.l1(l1_val),
            )
        )

    model.add(layers.BatchNormalization())

    if dropout > 0.0:
        model.add(layers.Dropout(dropout))

    # Binary classification
    model.add(
        layers.Dense(
            1,
            activation="sigmoid",
            kernel_initializer=initializers.get(init),
        )
    )

    lr = hp.Float("lr", 1e-5, 1e-2, sampling="log", default=1e-3)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(
        optimizer=opt,
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [32]:
# KERAS TUNER (HYPERBAND)


In [33]:
save_dir = os.path.join(TUNER_DIR, "kt_minimal")
os.makedirs(save_dir, exist_ok=True)

tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective("val_accuracy", direction="max"),
    max_epochs=15,
    directory=save_dir,
    project_name="minimal_project",
    overwrite=True,
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
)

print("\n🔎 Starting Hyperband search...")

tuner.search(
    X_train_transformed,
    y_train,
    epochs=15,
    validation_data=(X_val_transformed, y_val),
    batch_size=64,
    callbacks=[early_stop],
    verbose=1,
)


Trial 30 Complete [00h 01m 19s]
val_accuracy: 0.9256202578544617

Best val_accuracy So Far: 0.9564146995544434
Total elapsed time: 00h 17m 15s


In [34]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nBest hyperparameters:")
for k, v in best_hps.values.items():
    print(f"  {k}: {v}")

best_model = tuner.get_best_models(num_models=1)[0]


Best hyperparameters:
  units: 192
  dropout: 0.2
  l1: 9.142311834577992e-05
  init: glorot_uniform
  num_layers: 2
  lr: 0.0019962321212215937
  tuner/epochs: 15
  tuner/initial_epoch: 5
  tuner/bracket: 2
  tuner/round: 2
  tuner/trial_id: 0012


In [36]:
from sklearn.preprocessing import LabelEncoder

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [41]:
class_names = ["neutral or dissatisfied", "satisfied"]


y_proba = best_model.predict(X_test_transformed)


y_pred = (y_proba.ravel() >= 0.5).astype(int)


test_accuracy = accuracy_score(y_test, y_pred)


report = classification_report(
    y_test,
    y_pred,
    target_names=class_names,
    digits=4
)

#  Confusion matrix (optional but useful)
cm = confusion_matrix(y_test, y_pred)


print("\n====== TEST SET EVALUATION ======")
print(f"Accuracy : {test_accuracy:.4f}\n")

print("Classification Report:\n")
print(report)

print("Confusion Matrix :")
print(cm)

810/810 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

====== TEST SET EVALUATION ======
Accuracy : 0.9551

Classification Report:

                         precision    recall  f1-score   support

neutral or dissatisfied     0.9370    0.9656    0.9511     11721
              satisfied     0.9708    0.9463    0.9584     14177

               accuracy                         0.9551     25898
              macro avg     0.9539    0.9560    0.9548     25898
           weighted avg     0.9555    0.9551    0.9551     25898

Confusion Matrix :
[[11318   403]
 [  761 13416]]


In [45]:
PREPROCESSOR_PATH = os.path.join(MODEL_DIR, "preprocessor1.pkl")
MODEL_PATH = os.path.join(MODEL_DIR, "ann_model1.h5")


class_names = ["neutral or dissatisfied", "satisfied"]

preprocessing_bundle = {
    "preprocessor": preprocessor,
    "class_names": class_names,
    "numeric_features": numerical_features,
    "categorical_features": categorical_features,
}

# Save .pkl and .h5
joblib.dump(preprocessing_bundle, PREPROCESSOR_PATH)
best_model.save(MODEL_PATH)

print("\n💾 Saved preprocessing pipeline to:", PREPROCESSOR_PATH)
print("💾 Saved trained ANN model to     :", MODEL_PATH)



💾 Saved preprocessing pipeline to: /content/drive/MyDrive/Colab Notebooks/Deep Learning ANN/models/preprocessor1.pkl
💾 Saved trained ANN model to     : /content/drive/MyDrive/Colab Notebooks/Deep Learning ANN/models/ann_model1.h5
